# exp15

exp03 Tabnet entmax

https://www.kaggle.com/code/ragnar123/amex-lgbm-dart-cv-0-7977

In [1]:
# ====================================================
# Library
# ====================================================
import os
import gc
import warnings
warnings.filterwarnings('ignore')
import scipy as sp
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from tqdm.auto import tqdm
import itertools

import joblib
import random
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from tqdm.auto import tqdm
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostClassifier
from itertools import combinations

import ipywidgets as widgets
import snappy


from pytorch_tabnet.pretraining import TabNetPretrainer

from pytorch_tabnet.tab_model import TabNetRegressor
from pytorch_tabnet.tab_model import TabNetClassifier

import torch
from pytorch_tabnet.metrics import Metric
# from ipywidgets import interact, Select


# Training & Inference

In [2]:

# ====================================================
# Configurations
# ====================================================
class CFG:
    input_dir = '../feature/exp03_amex-fe/'
    output_dir = '../output/exp15 Tabnet entmax/'
    seed = 46
    n_folds = 5
    target = 'target'
    boosting_type = 'dart'
    metric = 'binary_logloss'
    model = "tabnet"
    ver = "exp15"

# ====================================================
# Seed everything
# ====================================================
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

# ====================================================
# Read data
# ====================================================
def read_data():
    train = pd.read_parquet(CFG.input_dir + 'train_fe_plus_plus.parquet')
    test = pd.read_parquet(CFG.input_dir + 'test_fe_plus_plus.parquet')
    return train, test

# ====================================================
# XGB train
# ====================================================

def cat_train(x, y, xt, yt,cat_features):
    print("# of features:", x.shape[1])
    assert x.shape[1] == xt.shape[1]
    
    prams = {
        'depth': 8,
        'iterations':5000,
#         'learning_rate': 0.05,
        'random_state':CFG.seed,
    }

#     watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
    watchlist = [(x, 'train'), (xt, 'eval')]
    clf = CatBoostClassifier(**prams)# 5000
    clf.fit(x, y, eval_set=[(xt, yt)], cat_features=cat_features,plot=True, verbose_eval = 100)
#     print('best ntree_limit:', clf.best_ntree_limit)
#     print('best score:', clf.best_score)
    # return clf.predict_proba(xt)[:, 1]
    return clf.predict_proba(xt)[:, 1], clf

def tabnet_train(x, y, xt, yt,cat_features):
    print("# of features:", x.shape[1])
    assert x.shape[1] == xt.shape[1]
    
    prams = {
        'depth': 8,
        'iterations':5000,
#         'learning_rate': 0.05,
        'random_state':CFG.seed,
    }

#     watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
    watchlist = [(x, 'train'), (xt, 'eval')]
    clf = CatBoostClassifier(**prams)# 5000
    clf.fit(x, y, eval_set=[(xt, yt)], cat_features=cat_features,plot=True, verbose_eval = 100)
#     print('best ntree_limit:', clf.best_ntree_limit)
#     print('best score:', clf.best_score)
    # return clf.predict_proba(xt)[:, 1]
    return clf.predict_proba(xt)[:, 1], clf



# using amex metric to evaluate tabnet
class Amex_tabnet(Metric):
    
    def __init__(self):
        self._name = 'amex_tabnet'
        self._maximize = True

    def __call__(self, y_true, y_pred):
        amex = amex_metric_numpy(y_true, y_pred[:, 1])
        return max(amex, 0.)

# ====================================================
# Amex metric
# ====================================================
def amex_metric(y_true, y_pred):
    labels = np.transpose(np.array([y_true, y_pred]))
    labels = labels[labels[:, 1].argsort()[::-1]]
    weights = np.where(labels[:,0]==0, 20, 1)
    cut_vals = labels[np.cumsum(weights) <= int(0.04 * np.sum(weights))]
    top_four = np.sum(cut_vals[:,0]) / np.sum(labels[:,0])
    gini = [0,0]
    for i in [1,0]:
        labels = np.transpose(np.array([y_true, y_pred]))
        labels = labels[labels[:, i].argsort()[::-1]]
        weight = np.where(labels[:,0]==0, 20, 1)
        weight_random = np.cumsum(weight / np.sum(weight))
        total_pos = np.sum(labels[:, 0] *  weight)
        cum_pos_found = np.cumsum(labels[:, 0] * weight)
        lorentz = cum_pos_found / total_pos
        gini[i] = np.sum((lorentz - weight_random) * weight)
    return 0.5 * (gini[1]/gini[0] + top_four)


def xgb_amex(y_pred, y_true):
    return 'amex', amex_metric_np(y_pred,y_true.get_label())


# Created by https://www.kaggle.com/yunchonggan
# https://www.kaggle.com/competitions/amex-default-prediction/discussion/328020
def amex_metric_np(preds: np.ndarray, target: np.ndarray) -> float:
    indices = np.argsort(preds)[::-1]
    preds, target = preds[indices], target[indices]

    weight = 20.0 - target * 19.0
    cum_norm_weight = (weight / weight.sum()).cumsum()
    four_pct_mask = cum_norm_weight <= 0.04
    d = np.sum(target[four_pct_mask]) / np.sum(target)

    weighted_target = target * weight
    lorentz = (weighted_target / weighted_target.sum()).cumsum()
    gini = ((lorentz - cum_norm_weight) * weight).sum()

    n_pos = np.sum(target)
    n_neg = target.shape[0] - n_pos
    gini_max = 10 * n_neg * (n_pos + 20 * n_neg - 19) / (n_pos + 20 * n_neg)

    g = gini / gini_max
    return 0.5 * (g + d)

def amex_metric_numpy(y_true: np.array, y_pred: np.array) -> float:

    # count of positives and negatives
    n_pos = y_true.sum()
    n_neg = y_true.shape[0] - n_pos

    # sorting by descring prediction values
    indices = np.argsort(y_pred)[::-1]
    preds, target = y_pred[indices], y_true[indices]

    # filter the top 4% by cumulative row weights
    weight = 20.0 - target * 19.0
    cum_norm_weight = (weight / weight.sum()).cumsum()
    four_pct_filter = cum_norm_weight <= 0.04

    # default rate captured at 4%
    d = target[four_pct_filter].sum() / n_pos

    # weighted gini coefficient
    lorentz = (target / n_pos).cumsum()
    gini = ((lorentz - cum_norm_weight) * weight).sum()

    # max weighted gini coefficient
    gini_max = 10 * n_neg * (1 - 19 / (n_pos + 20 * n_neg))

    # normalized weighted gini coefficient
    g = gini / gini_max

    return 0.5 * (g + d)

In [3]:
seed_everything(CFG.seed)
train, test = read_data()


In [4]:
# Get feature list
features = [col for col in train.columns if col not in ['customer_ID', CFG.target]]
# all_data = features.extend("customer_ID")


train.fillna(value=0, inplace=True)
test.fillna(value=0, inplace=True)

## infを含むデータを外れ値（－１００００）に置換
train = train.replace([np.inf, -np.inf],1000000000)
test = test.replace([np.inf, -np.inf],1000000000)

# Create a numpy array to store test predictions
test_predictions = np.zeros(len(test))
# Create a numpy array to store out of folds predictions
oof_predictions = np.zeros(len(train))


preds = []
cids = []
tr_target = []
test_preds = []

kfold = StratifiedKFold(n_splits = CFG.n_folds, shuffle = True, random_state = CFG.seed)
for fold, (trn_ind, val_ind) in enumerate(kfold.split(train, train[CFG.target])):
    print(' ')
    print('-'*50)
    print(f'Training fold {fold} with {len(features)} features...')
    
    if fold == 0 or fold == 1,:
        print("pass training fold")
        continue
    
    x_train, x_val = train[features].iloc[trn_ind], train[features].iloc[val_ind]
    y_train, y_val = train[CFG.target].iloc[trn_ind], train[CFG.target].iloc[val_ind]
    
#     cid = train["customer_ID"].loc[val_ind]
    cid = train["customer_ID"].loc[val_ind]
    cids.extend(cid)
    # lgb_train = lgb.Dataset(x_train, y_train, categorical_feature = cat_features)
    # lgb_valid = lgb.Dataset(x_val, y_val, categorical_feature = cat_features)

    
    model = TabNetClassifier(n_d = 32,
                         n_a = 32,
                         n_steps = 3,
                         gamma = 1.3,
                         n_independent = 2,
                         n_shared = 2,
                         momentum = 0.02,
                         clip_value = None,
                         lambda_sparse = 1e-3,
                         optimizer_fn = torch.optim.Adam,
                         optimizer_params = dict(lr = 1e-3, weight_decay=1e-3),
                         scheduler_fn = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts,
                         scheduler_params = {'T_0':5,
                                             'eta_min':1e-4,
                                             'T_mult':1,
                                             'last_epoch':-1},
                         mask_type = 'entmax',
                         seed = CFG.seed)

    print(f'start Training fold {fold}')
    model.fit(np.array(x_train),
                    np.array(y_train.values.ravel()),
                    eval_set = [(np.array(x_val), np.array(y_val.values.ravel()))],
                    max_epochs = 50,#50
                    patience = 50,#50
                    batch_size = 512,
                    eval_metric = ['auc', 'accuracy', Amex_tabnet]) # Last metric is used for early stoppin
    
    
    file_path = f"{CFG.output_dir}{CFG.model}_fold{fold}"
    saved_filepath = model.save_model(file_path)
    
#     oof_predictions[valid_idx] = model.predict_proba(x_val.values)[:, 1]
    pred = model.predict_proba(np.array(x_val))[:, 1]
    preds.extend(pred)
    tr_target.extend(y_val)
    
    fold_score = amex_metric(y_val, pred)
    print(f"{fold} score is : ",fold_score)
    
    # OOF用のcustomer_IDをExtend
    cids.extend(train["customer_ID"].loc[val_ind])
    test_preds.append(model.predict_proba(np.array(test[features]))[:, 1])
    # test predct
#     print(f'OOF score across folds: {amex_metric_numpy(target, oof_predictions.flatten())}')

#     # Compute fold metric
    del x_train, x_val, y_train, y_val
    gc.collect()


 
--------------------------------------------------
Training fold 0 with 1460 features...
pass training fold
 
--------------------------------------------------
Training fold 1 with 1460 features...
pass training fold
 
--------------------------------------------------
Training fold 2 with 1460 features...
pass training fold
 
--------------------------------------------------
Training fold 3 with 1460 features...
start Training fold 3
epoch 0  | loss: 0.53533 | val_0_auc: 0.91519 | val_0_accuracy: 0.84288 | val_0_amex_tabnet: 0.61936 |  0:04:49s
epoch 1  | loss: 0.36586 | val_0_auc: 0.92877 | val_0_accuracy: 0.86212 | val_0_amex_tabnet: 0.66351 |  0:09:39s
epoch 2  | loss: 0.32232 | val_0_auc: 0.93464 | val_0_accuracy: 0.86606 | val_0_amex_tabnet: 0.6854  |  0:14:29s
epoch 3  | loss: 0.30048 | val_0_auc: 0.93859 | val_0_accuracy: 0.87247 | val_0_amex_tabnet: 0.70112 |  0:19:17s
epoch 4  | loss: 0.28998 | val_0_auc: 0.94057 | val_0_accuracy: 0.87384 | val_0_amex_tabnet: 0.70878 |  0

In [5]:

############## ここにFold1の予測結果を読み込む処理を書く










# Compute out of folds metric
test_predictions = np.mean(test_preds,axis = 0)
score = amex_metric(train[CFG.target], preds)
print(f'Our out of folds CV score is {score}')

# Create a dataframe to store test prediction
test_df = pd.DataFrame({'customer_ID': test['customer_ID'], 'prediction': test_predictions})
# test_df.to_csv(f'{CFG.output_dir}test_{CFG.model}_{score}_baseline_{CFG.n_folds}fold_seed{CFG.seed}.csv', index = False)
test_df.to_csv(f'{CFG.output_dir}test_{CFG.model}_{score}_{CFG.n_folds}fold_seed{CFG.seed}.csv', index = False)


dic_oof = {
    "customer_ID":cids,
    "target":tr_target,
    "tabnet_oot":oof_predictions
}

# Create a dataframe to store out of folds predictions
oof_df = pd.DataFrame(dic_oof)
# oof_df = pd.DataFrame({'customer_ID': train['customer_ID'], 'target': train[CFG.target], 'prediction': oof_predictions})
oof_df.to_csv(f'{CFG.output_dir}oof_{CFG.ver}_{CFG.model}_{score}_{CFG.n_folds}fold_seed{CFG.seed}.csv', index = False)



IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

# Infer

学習しない場合こちら

In [4]:

# Get feature list
features = [col for col in train.columns if col not in ['customer_ID', CFG.target]]
# all_data = features.extend("customer_ID")


train.fillna(value=0, inplace=True)
test.fillna(value=0, inplace=True)

## infを含むデータを外れ値（－１００００）に置換
train = train.replace([np.inf, -np.inf],1000000000)
test = test.replace([np.inf, -np.inf],1000000000)

# Create a numpy array to store test predictions
test_predictions = np.zeros(len(test))
# Create a numpy array to store out of folds predictions
oof_predictions = np.zeros(len(train))


preds = []
cids = []
tr_target = []
test_preds = []


kfold = StratifiedKFold(n_splits = CFG.n_folds, shuffle = True, random_state = CFG.seed)
for fold, (trn_ind, val_ind) in enumerate(kfold.split(train, train[CFG.target])):

    print(f"fold : {fold}")
    
    x_train, x_val = train[features].iloc[trn_ind], train[features].iloc[val_ind]
    y_train, y_val = train[CFG.target].iloc[trn_ind], train[CFG.target].iloc[val_ind]
    
    model = TabNetClassifier(n_d = 32,
                         n_a = 32,
                         n_steps = 3,
                         gamma = 1.3,
                         n_independent = 2,
                         n_shared = 2,
                         momentum = 0.02,
                         clip_value = None,
                         lambda_sparse = 1e-3,
                         optimizer_fn = torch.optim.Adam,
                         optimizer_params = dict(lr = 1e-3, weight_decay=1e-3),
                         scheduler_fn = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts,
                         scheduler_params = {'T_0':5,
                                             'eta_min':1e-4,
                                             'T_mult':1,
                                             'last_epoch':-1},
                         mask_type = 'entmax',
                         seed = CFG.seed)
    
    print("load model")
    model_path = f"../output/exp15 Tabnet entmax/tabnet_fold{fold}.zip"
    model.load_model(model_path)
    
    cid = train["customer_ID"].loc[val_ind]
    cids.extend(cid)
    
    pred = model.predict_proba(np.array(x_val))[:, 1]
    preds.extend(pred)
    
    fold_score = amex_metric(y_val, pred)
    print(f"{fold} score is : ",fold_score)
    
    test_pred= model.predict_proba(np.array(test[features]))[:, 1]
    test_preds.append(test_pred)
    
    tr_target.extend(y_val)
#     pred = model.predict_proba(test[all_features].values)[:, 1]

    del pred,cid
    


fold : 0
load model
0 score is :  0.7920919714728045
fold : 1
load model
1 score is :  0.7876776860325015
fold : 2
load model
2 score is :  0.7890420759417227
fold : 3
load model
3 score is :  0.7888245459171852
fold : 4
load model
4 score is :  0.7876384784395436


In [8]:
test_preds

[array([0.01365403, 0.00291401, 0.03214177, ..., 0.5378061 , 0.33354393,
        0.09402709], dtype=float32),
 array([0.01883214, 0.00218072, 0.04229571, ..., 0.44275218, 0.22777435,
        0.08200692], dtype=float32),
 array([0.01187868, 0.00250312, 0.03653168, ..., 0.66964495, 0.3351964 ,
        0.11270957], dtype=float32),
 array([0.02224178, 0.00230813, 0.02615114, ..., 0.67060965, 0.29399443,
        0.07807986], dtype=float32),
 array([0.03113907, 0.00377897, 0.04021817, ..., 0.55600375, 0.3859081 ,
        0.08209409], dtype=float32)]

In [16]:

# Compute out of folds metric
test_predictions = np.mean(test_preds,axis = 0)
# score = amex_metric(train[CFG.target], preds)
score = amex_metric(tr_target, preds)

print(f'Our out of folds CV score is {score}')

# Create a dataframe to store test prediction
test_predictions = np.mean(test_preds,axis = 0)

test_df = pd.DataFrame({'customer_ID': test['customer_ID'], 'prediction': test_predictions})
# test_df.to_csv(f'{CFG.output_dir}test_{CFG.model}_{score}_baseline_{CFG.n_folds}fold_seed{CFG.seed}.csv', index = False)
test_df.to_csv(f'{CFG.output_dir}test_{CFG.model}_{score}_{CFG.n_folds}fold_seed{CFG.seed}.csv', index = False)


dic_oof = {
    "customer_ID":cids,
#     "target":tr_target,
    "tabnet_oof":preds
}

# Create a dataframe to store out of folds predictions
oof_df = pd.DataFrame(dic_oof)
# oof_df = pd.DataFrame({'customer_ID': train['customer_ID'], 'target': train[CFG.target], 'prediction': oof_predictions})
oof_df.to_csv(f'{CFG.output_dir}oof_{CFG.ver}_{CFG.model}_{score}_{CFG.n_folds}fold_seed{CFG.seed}.csv', index = False)








# sub = np.array(preds).mean(axis = 0)
# df_sub = pd.read_csv('../input/amex-default-prediction/sample_submission.csv')
# df_sub['prediction'] = sub

# df_sub.to_csv('tabnet_pred.csv',index = False)


Our out of folds CV score is 0.788206478467055


In [17]:
test_df.head()

,customer_ID,prediction
0,00000469ba478561f23a92a868bd366de6f6527a684c9a...,0.019549
1,00001bf2e77ff879fab36aa4fac689b9ba411dae63ae39...,0.002737
2,0000210045da4f81e5f122c6bde5c2a617d03eef67f82c...,0.035468
3,00003b41e58ede33b8daf61ab56d9952f17c9ad1c3976c...,0.262243
4,00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9...,0.708322


In [18]:
oof_df.head()

,customer_ID,tabnet_oof
0,00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8a...,0.002710
1,00031e8be98bc3411f6037cbd4d3eeaf24b3ae221682b7...,0.017477
2,0003e58375faf90552b2861c1ccea4ee5757827cbb6ecd...,0.009180
3,0004837f0c785928a29a6f83f70f4a1c54caec483a773f...,0.228023
4,0005e52a3fa31b7eed49ceb576f011433ee2578833cd3f...,0.779503
